## How to predict a single image on a Pre-trained Model - Resnet34

### Importing Libraries

In [166]:
import torch
from torchvision import models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import torch.optim as optim
import torch.nn as nn

### Loading Pretrained Model

In [2]:
resnet = models.resnet34(pretrained=True)

### Creating Transforms

In [3]:
preprocess= transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.5, 0.5, 0.5],
        std = [0.2, 0.2, 0.2])
    ])

### Load your Single File

In [4]:
img = Image.open('traffic.jpeg')

In [5]:
img_p = preprocess(img)

In [6]:
img_p.shape

torch.Size([3, 224, 224])

### Model requires batch size as first dimension, so we reshape the image dimension.

In [7]:
batch_t = torch.unsqueeze(img_p, 0)

In [8]:
batch_t.shape

torch.Size([1, 3, 224, 224])

**Since we are predicting using pretrained model, we use model under eval mode. 
And in eval mode, the batchNorm and Dropout layers of the model will be disabled. 
The out variable contains our predicted output over 1000 classes.**

In [9]:
resnet.eval()
out = resnet(batch_t)
print(out.shape)

torch.Size([1, 1000])


### Loading Images Classes from txt file

In [10]:
with open('imagenet_class.txt') as f:
    classes = [line.strip().split(",")[1].strip() for line in f.readlines()]

### Finding Index of the max probability class

In [11]:
_, index = torch.max(out, 1)

### Confidence of Prediction

In [12]:
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
classes[index[0]], percentage[index[0]].item()

('traffic_light', 99.99995422363281)

### Top 5 predictions

In [13]:
_, indices = torch.sort(out, descending=True)
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('traffic_light', 99.99995422363281),
 ('street_sign', 2.8018390366923995e-05),
 ('pole', 9.717282409837935e-06),
 ('loudspeaker', 2.9554805678344565e-06),
 ('binoculars', 1.4750306718269712e-06)]

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [15]:
img_t.shape

torch.Size([3, 5, 5])

In [16]:
weights.shape

torch.Size([3])

In [17]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [18]:
batch_t.shape

torch.Size([2, 3, 5, 5])

In [19]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [20]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)

In [21]:
unsqueezed_weights

tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]])

In [22]:
img_weights = (img_t * unsqueezed_weights)

In [23]:
img_weights.shape

torch.Size([3, 5, 5])

In [24]:
batch_weights = (batch_t * unsqueezed_weights)

In [25]:
batch_weights.shape

torch.Size([2, 3, 5, 5])

In [26]:
img_gray_weighted = img_weights.sum(-3)

In [27]:
img_gray_weighted

tensor([[-0.4142, -0.2200, -0.2374, -1.1039,  0.3578],
        [ 0.0676,  0.1268,  0.9222, -1.0375,  0.2899],
        [ 1.3845,  0.1224, -0.8231, -0.3893, -0.1400],
        [-0.0285,  0.7510,  0.7330, -0.2020, -1.3866],
        [-0.1159, -0.0665, -1.3296,  1.7022,  0.0150]])

In [28]:
batch_gray_weighted = batch_weights.sum(-3)

In [29]:
batch_gray_weighted.shape

torch.Size([2, 5, 5])

In [30]:
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [31]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [32]:
img_gray_weighted_fancy

tensor([[-0.4142, -0.2200, -0.2374, -1.1039,  0.3578],
        [ 0.0676,  0.1268,  0.9222, -1.0375,  0.2899],
        [ 1.3845,  0.1224, -0.8231, -0.3893, -0.1400],
        [-0.0285,  0.7510,  0.7330, -0.2020, -1.3866],
        [-0.1159, -0.0665, -1.3296,  1.7022,  0.0150]])

### Named Tensor

In [33]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


In [34]:
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [35]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [36]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [37]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [38]:
#gray_named = (img_named[..., :3] * weights_named).sum('channels')

In [39]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

### Tensor Storage

In [40]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [41]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

### Inplace replacement using _

In [42]:
a = torch.ones(3, 2)

In [43]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### Playing with Storage

In [44]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [45]:
points[1]

tensor([5., 3.])

In [46]:
a = list(range(9))

In [47]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [48]:
a = torch.tensor(a)

In [49]:
a.size()

torch.Size([9])

In [50]:
a.stride()

(1,)

In [51]:
a.view(3,3)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [52]:
b = a.view(3,3)

In [53]:
id(a.storage)==id(b.storage)

True

In [54]:
c = b[1:, 1:]

In [55]:
c

tensor([[4, 5],
        [7, 8]])

### Working with Images

In [56]:
import imageio

In [57]:
image_arr = imageio.imread('traffic.jpeg')

In [58]:
image_arr.shape

(259, 194, 3)

Pytorch format for image C * H * W

In [59]:
img_tensor = torch.from_numpy(image_arr)

In [60]:
img_tensor_chw = img_tensor.permute(2, 0, 1)

In [61]:
img_tensor_chw.shape

torch.Size([3, 259, 194])

### Including Batch 1 at different Position

In [62]:
img_tensor_chw.unsqueeze(0).shape

torch.Size([1, 3, 259, 194])

In [63]:
img_tensor_chw.unsqueeze(1).shape

torch.Size([3, 1, 259, 194])

In [64]:
img_tensor_chw.unsqueeze(-1).shape

torch.Size([3, 259, 194, 1])

### Efficient Way

In [65]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.int16)

Sometimes images also have an alpha channel
indicating transparency.

In [66]:
data = '/home/mayur/Desktop/Kaggle Notebooks/Generative Dog Images/all-dogs'

In [67]:
filenames = [name for name in os.listdir(data) if os.path.splitext(name)[-1] == '.jpg']

In [68]:
pil_img_25 = [Image.open(os.path.join(data, f)) for f in filenames[:25]]

In [69]:
pil_transform = transforms.Compose([transforms.Resize((256, 256))])

f = lambda: [pil_transform(img) for img in pil_img_25]

In [70]:
for i, file in enumerate(f()):
    if i == 3:
        break
    file = np.asarray(file)
    file = torch.from_numpy(file).permute(2, 0, 1)
    batch[i] = file

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [71]:
batch.shape

torch.Size([3, 3, 256, 256])

### Normalize

#### One way to normalize is image/=255.0

In [72]:
### Second way to Normalize
batch = batch.float()

n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

### One Hot Encoding in Torch

In [73]:
target = torch.randint(1, 5, (25,))

In [74]:
target_onehot = torch.zeros(target.shape[0], 5)

In [75]:
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.]])

In [76]:
target_unsqueezed = target.unsqueeze(1)

In [77]:
target_unsqueezed

tensor([[1],
        [1],
        [4],
        [1],
        [3],
        [3],
        [1],
        [1],
        [1],
        [4],
        [4],
        [4],
        [4],
        [2],
        [3],
        [1],
        [2],
        [2],
        [4],
        [1],
        [4],
        [3],
        [1],
        [4],
        [2]])

In [78]:
data = torch.randn(25, 10)

In [79]:
d_mean = torch.mean(data, dim=0)

In [80]:
d_var = torch.var(data, dim=0)

In [81]:
data_normalized = (data - d_mean) / torch.sqrt(d_var)

In [82]:
data_normalized.shape

torch.Size([25, 10])

In [83]:
bad_index = target<=2

In [84]:
bad_data = data[bad_index]

In [85]:
bad_data.shape

torch.Size([13, 10])

### Time Series using Bike Sharing dataset

In [86]:
# 2 Years Data
bikes_numpy = np.loadtxt("data/hour-fixed.csv",
dtype=np.float32,
delimiter=",",
skiprows=1,
converters={1: lambda x: float(x[8:10])})

In [87]:
bikes_numpy.shape

(17520, 17)

In [88]:
bikes = torch.from_numpy(bikes_numpy)

In [89]:
bikes.stride()

(17, 1)

In [90]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

In [91]:
daily_bikes.shape

torch.Size([730, 24, 17])

In [92]:
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))


Stride : 24 * 17 = 408

N: 730 days

In [93]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [94]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [95]:
weather_onehot.scatter_(
dim=1,
index=first_day[:,9].unsqueeze(1).long() - 1,
value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [96]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [97]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
daily_bikes.shape[2])

In [98]:
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [99]:
daily_weather_onehot[0]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [100]:
daily_weather_onehot.scatter_(
1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [101]:
daily_weather_onehot[0]

tensor([[1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
         0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])

In [102]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [103]:
daily_bikes.shape

torch.Size([730, 21, 24])

In [104]:
daily_bikes[:, 9, :][0]

tensor([1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
        3., 3., 2., 2., 2., 2.])

In [105]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [106]:
daily_bikes[0].shape

torch.Size([21, 24])

### Keeping Temp values between [0, 1] with standardization

In [107]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
/ (temp_max - temp_min))

In [108]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
/ torch.std(temp))

### Working with Text using pytorch

### Character Level Conversion

In [109]:
with open("data/anna.txt", encoding='utf-8') as f:
    text = f.read()

In [110]:
lines = text.split("\n")

In [111]:
line = lines[100]

In [112]:
letter_t = torch.zeros(len(line), 128)

In [113]:
letter_t.shape

torch.Size([29, 128])

In [114]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    #print(letter_index)
    letter_t[i][letter_index] = 1
    

### Word Level Conversion

In [115]:
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n',' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

In [116]:
words_in_line = clean_words(line)

In [117]:
line, words_in_line

('despair, and found no answer.', ['despair', 'and', 'found', 'no', 'answer'])

In [118]:
word_list = sorted(set(clean_words(text)))

In [119]:
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

In [120]:
len(word2index_dict.keys())

15070

In [121]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))

In [122]:
word_t.shape

torch.Size([5, 15070])

In [123]:
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

 0 3588 despair
 1  680 and
 2 5362 found
 3 8926 no
 4  732 answer


### How To Model

In [124]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [125]:
def model(t_u, w, b):
    return w*t_u + b

In [126]:
def loss(t_p, t_c):
    return torch.mean((t_p-t_c)**2)

In [127]:
w = torch.ones(())
b = torch.zeros(())

In [128]:
t_p = model(w, t_u, b)

In [129]:
Loss = loss(t_p, t_c)

In [130]:
Loss

tensor(1763.8846)

In [131]:
delta = 0.1

loss_rate_of_change_w = \
(loss(model(t_u, w + delta, b), t_c) -
loss(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

In [132]:
loss_rate_of_change_w

tensor(4517.2979)

In [133]:
#(loss(model(t_u, w + delta, b), t_c) - loss(model(t_u, w - delta, b), t_c)) / (2 * delta)

In [134]:
learning_rate = 1e-2

In [135]:
w = w - learning_rate * loss_rate_of_change_w

In [136]:
loss_rate_of_change_b = \
(loss(model(t_u, w, b + delta), t_c) -
loss(model(t_u, w, b - delta), t_c)) / (2.0 * delta)
b = b - learning_rate * loss_rate_of_change_b

### Computing Derivative

d loss / d w = (d loss / d t_p) * (d t_p / d w)

In [137]:
def d_loss(t_p, t_c):
    dsq_diff = 2 * (t_p - t_c) / t_p.size(0)
    
    return dsq_diff

In [138]:
#### if d(w*t_u + b)/ dw = t_u + 0 = t_u

def d_model_dw(w, t_u, b):
    return t_u 

In [139]:
#### if d(w*t_u + b)/ db = 0 + 1.0 = 1.0

def d_model_db(w, t_u, b):
    return 1.0

In [140]:
def grad_fn(t_u, t_c, t_p, w, b):
    
    dloss_dtp = d_loss(t_p, t_c)
    dloss_dw = dloss_dtp * d_model_dw(t_u, w, b)
    dloss_db = dloss_dtp * d_model_db(t_u, w, b)
    

    return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [141]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        w, b = params
        t_p = model(t_u, w, b) #Forward pass
        
        Loss = loss(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b) #Backward pass
        
        params = params - learning_rate * grad
        if epoch % 10 == 1:
            print('Epoch %d, Loss %f' % (epoch, float(Loss)))
            print(f'params: {params}')
            print(f'grad: {grad}')
    return params

In [142]:
params = training_loop(
n_epochs = 100,
learning_rate = 1e-2,
params = torch.tensor([1.0, 0.0]),
t_u = t_u,
t_c = t_c)

Epoch 1, Loss 1763.884644
params: tensor([ 0.1740, -0.8260])
grad: tensor([82.6000, 82.6000])
Epoch 11, Loss 30.432922
params: tensor([ 0.2113, -0.6289])
grad: tensor([-0.1006, -0.4782])
Epoch 21, Loss 29.714087
params: tensor([ 0.2143, -0.6147])
grad: tensor([-0.0065, -0.0305])
Epoch 31, Loss 29.671791
params: tensor([ 0.2145, -0.6138])
grad: tensor([-0.0004, -0.0019])
Epoch 41, Loss 29.669165
params: tensor([ 0.2146, -0.6137])
grad: tensor([-2.5302e-05, -1.1790e-04])
Epoch 51, Loss 29.669001
params: tensor([ 0.2146, -0.6137])
grad: tensor([-1.8179e-06, -8.5831e-06])
Epoch 61, Loss 29.668993
params: tensor([ 0.2146, -0.6137])
grad: tensor([-2.9802e-07, -1.6689e-06])
Epoch 71, Loss 29.668993
params: tensor([ 0.2146, -0.6137])
grad: tensor([-2.9802e-07, -1.6689e-06])
Epoch 81, Loss 29.668993
params: tensor([ 0.2146, -0.6137])
grad: tensor([-2.9802e-07, -1.6689e-06])
Epoch 91, Loss 29.668993
params: tensor([ 0.2146, -0.6137])
grad: tensor([-2.9802e-07, -1.6689e-06])


### Autograd for derivation !

In [143]:
params = torch.tensor([1.0, 0.0],requires_grad=True)

In [144]:
params.grad is None

True

In [145]:
Loss = loss(model(t_u, *params), t_c)

In [146]:
Loss.backward()

In [147]:
params.grad

tensor([4517.2969,   82.6000])

In [148]:
if params.grad is not None:
    params.grad.zero_()

In [149]:
def training_loop_AG(n_epochs, learning_rate, params, t_u, t_c):
    
    for epoch in range(1, n_epochs+1):
        
        if params.grad is not None:
            params.grad.zero_()
            
        t_p = model(t_u, *params)
        Loss = loss(t_p, t_c)
        Loss.backward()
        
        with torch.no_grad():
            params -= learning_rate * params.grad
            
        if epoch % 100 == 0:
            print(f'epoch {epoch}, Loss: {Loss}')
            
    return params

In [150]:
t_un = 0.1 * t_u #scaling down

In [151]:
training_loop_AG(
n_epochs = 1000,
learning_rate = 1e-2,
params = torch.tensor([1.0, 0.0], requires_grad=True),
t_u = t_un,
t_c = t_c)

epoch 100, Loss: 22.148710250854492
epoch 200, Loss: 16.608064651489258
epoch 300, Loss: 12.664560317993164
epoch 400, Loss: 9.857802391052246
epoch 500, Loss: 7.8601155281066895
epoch 600, Loss: 6.438284397125244
epoch 700, Loss: 5.426309585571289
epoch 800, Loss: 4.706046104431152
epoch 900, Loss: 4.1934051513671875
epoch 1000, Loss: 3.828537940979004


tensor([  4.8021, -14.1031], requires_grad=True)

### Optimizer

In [152]:
def training_loop_optim(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs+1):
        
        t_p = model(t_u, *params)
        Loss = loss(t_p, t_c)
        optimizer.zero_grad()
        
        Loss.backward()
        optimizer.step()
        
        if epoch % 100 == 0:
            print(f'Epoch: {epoch}, Loss: {Loss}')
        
    return params

In [153]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

In [154]:
training_loop_optim(n_epochs=500, optimizer=optimizer,params= params,  t_u = t_un,t_c= t_c)

Epoch: 100, Loss: 22.148710250854492
Epoch: 200, Loss: 16.608068466186523
Epoch: 300, Loss: 12.664565086364746
Epoch: 400, Loss: 9.857809066772461
Epoch: 500, Loss: 7.8601179122924805


tensor([ 4.0443, -9.8133], requires_grad=True)

### Splitting Dataset into Train and Validation

In [155]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([ 0,  6, 10,  2,  3,  8,  5,  9,  1]), tensor([4, 7]))

In [156]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]
val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [157]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss(train_t_p, train_t_c)
        val_t_p = model(val_t_u, *params)
        val_loss = loss(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                    f" Validation loss {val_loss.item():.4f}")
    return params

In [158]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

In [159]:
training_loop(
n_epochs = 500,
optimizer = optimizer,
params = params,
train_t_u = train_t_un,
val_t_u = val_t_un,
train_t_c = train_t_c,
val_t_c = val_t_c)

Epoch 1, Training loss 90.7754, Validation loss 33.5146
Epoch 2, Training loss 33.7982, Validation loss 34.4394
Epoch 3, Training loss 27.0283, Validation loss 42.0913
Epoch 100, Training loss 21.0966, Validation loss 35.6010
Epoch 200, Training loss 17.0360, Validation loss 26.2173
Epoch 300, Training loss 13.8714, Validation loss 19.2201
Epoch 400, Training loss 11.4051, Validation loss 14.0459
Epoch 500, Training loss 9.4831, Validation loss 10.2596


tensor([ 3.9074, -8.6521], requires_grad=True)

In [160]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss(train_t_p, train_t_c)
        
        with torch.no_grad():
            val_t_p = model(val_t_u, *params)
            val_loss = loss(val_t_p, val_t_c)
            assert val_loss.requires_grad == False

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
    return params

### Set Grad Enabled - Acts like switch to ON and OFF the autograd

In [161]:
def calc_forward(t_u, t_c, is_train):
    with torch.set_grad_enabled(is_train):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
    return loss

### Model - w2 * t_u ** 2 + w1 * t_u + b

In [162]:
params = torch.randn(3,) 
params.requires_grad=True
criterion = torch.nn.MSELoss()
optimizer = optim.Adam([params], lr=1e-2)

In [163]:
def new_model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [230]:
def train(n_epochs, optimizer, optimizer, train_t_u, val_t_u,train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        w1, w2, b = params
        train_t_p = new_model(train_t_u, w1, w2, b)
        train_loss = criterion(train_t_p, train_t_c)
        
        with torch.no_grad():
            val_t_p = new_model(val_t_u, *params)
            val_loss = criterion(val_t_p, val_t_c)
            assert val_loss.requires_grad == False

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        if epoch % 100==0:
            print(f'Epoch: {epoch}, Loss: {train_loss}')
            print(params)
        
    return params

SyntaxError: duplicate argument 'optimizer' in function definition (<ipython-input-230-73a45e2719d4>, line 4)

In [165]:
train(n_epochs = 500,
optimizer = optimizer,
params=params,
train_t_u = train_t_un,
val_t_u = val_t_un,
train_t_c = train_t_c,
val_t_c = val_t_c)

Epoch: 100, Loss: 4.886602401733398
tensor([ 0.4475,  0.3748, -1.6703], requires_grad=True)
Epoch: 200, Loss: 3.933260679244995
tensor([ 0.2843,  0.4087, -2.0471], requires_grad=True)
Epoch: 300, Loss: 3.2933266162872314
tensor([ 0.1325,  0.4411, -2.3996], requires_grad=True)
Epoch: 400, Loss: 2.9597949981689453
tensor([ 0.0151,  0.4665, -2.6855], requires_grad=True)
Epoch: 500, Loss: 2.8161251544952393
tensor([-0.0629,  0.4840, -2.8981], requires_grad=True)


tensor([-0.0629,  0.4840, -2.8981], requires_grad=True)

### Developing Neural Nets Using nn.Module

**Model with one sample with one feature**

In [201]:
x = torch.ones(1)

In [172]:
l_model = nn.Linear(1,1)

In [173]:
l_model(x)

tensor([-1.4953], grad_fn=<AddBackward0>)

In [178]:
l_model.weight

Parameter containing:
tensor([[-0.7649]], requires_grad=True)

In [179]:
l_model.bias

Parameter containing:
tensor([-0.7304], requires_grad=True)

In [202]:
x.shape

torch.Size([1])

In [206]:
x_add_d = x.unsqueeze(1)

In [207]:
x_add_d.shape

torch.Size([1, 1])

**Model which takes batch of samples with one feature**

In [208]:
x = torch.ones(10, 1)

In [209]:
x

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [210]:
x.shape

torch.Size([10, 1])

In [200]:
l_model(x)

tensor([[-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953],
        [-1.4953]], grad_fn=<AddmmBackward>)

### Building Neural Network with One Hidden Layer

In [218]:
x = torch.tensor(t_u).unsqueeze(1)
y = torch.tensor(t_c).unsqueeze(1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [219]:
seq_model = nn.Sequential(
    nn.Linear(1, 10),
    nn.Tanh(),
    nn.Linear(10, 1)
    )

In [221]:
[p.shape for p in seq_model.parameters()]

[torch.Size([10, 1]), torch.Size([10]), torch.Size([1, 10]), torch.Size([1])]

In [222]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([10, 1])
0.bias torch.Size([10])
2.weight torch.Size([1, 10])
2.bias torch.Size([1])


In [233]:
from collections import OrderedDict
seq_model = nn.Sequential(OrderedDict([
('hidden_linear', nn.Linear(1, 9)),
('hidden_activation', nn.Tanh()),
('output_linear', nn.Linear(9, 1))
]))

In [234]:
seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=9, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=9, out_features=1, bias=True)
)

In [235]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([9, 1])
hidden_linear.bias torch.Size([9])
output_linear.weight torch.Size([1, 9])
output_linear.bias torch.Size([1])


In [236]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.3329], requires_grad=True)

In [237]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
            f" Validation loss {loss_val.item():.4f}")

In [242]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
n_epochs = 5000,
optimizer = optimizer,
model = seq_model,
loss_fn = nn.MSELoss(),
t_u_train = train_t_un.unsqueeze(1),
t_u_val = val_t_un.unsqueeze(1),
t_c_train = train_t_c.unsqueeze(1),
t_c_val = val_t_c.unsqueeze(1))

print('output', seq_model(val_t_un.unsqueeze(1)))
print('answer', val_t_c.unsqueeze(1))
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 1.8214, Validation loss 6.1820
Epoch 1000, Training loss 1.8137, Validation loss 6.3981
Epoch 2000, Training loss 1.8061, Validation loss 6.6104
Epoch 3000, Training loss 1.7986, Validation loss 6.8202
Epoch 4000, Training loss 1.7910, Validation loss 7.0274
Epoch 5000, Training loss 1.7834, Validation loss 7.2324
output tensor([[12.5741],
        [-0.5377]], grad_fn=<AddmmBackward>)
answer tensor([[11.],
        [-4.]])
hidden tensor([[ 2.8302e-03],
        [ 8.7560e-03],
        [-1.5450e-02],
        [ 8.4116e-03],
        [-9.8095e-03],
        [-2.1610e-03],
        [ 3.1948e-05],
        [ 1.0729e-04],
        [ 1.5020e-03]])
